In [ ]:
%matplotlib inline
from IPython.display import IFrame
from answers import * # Ответы и чекеры

# Пересечение прямоугольника с множеством непересекающихся отрезков

## Мотивация
Для решения задач вычислительной геометрии часто как вспомогательный прием используется метод заключения объекта на плоскости в прямоугольник минимального размера (англ. bounding box), стороны которого параллельны осям системы координат (англ. axis-aligned).
В связи с этим полезным становится уметь решать следующую задачу.

### Задача 
Дано множество непересекающихся отрезков в $\Bbb R^{2}$ (см. рис.1).
Нужно уметь отвечать на запросы, какие из них пересекают границу данного axis-aligned прямоугольника. 

![](img/rectg.svg)

## Дерево отрезков (англ. [Segment tree](https://en.wikipedia.org/wiki/Segment_tree))

Для решения поставленной задачи воспользуемся структурой данных, называющейся деревом отрезков.
Чтобы избежать путаницы из-за русскоязычной терминологии, сразу обратим внимание, что речь идет не о том [дереве отрезков](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D1%80%D0%B5%D0%B2%D0%BE_%D0%BE%D1%82%D1%80%D0%B5%D0%B7%D0%BA%D0%BE%D0%B2), позволяющем эффективно отвечать на запросы вычисления некоторой ассоциативной функции на отрезке, которое рассматривают в классическом курсе структур данных.

Для начала рассмотрим вещественную ось, структура данных должна уметь отвечать на запросы о поиске отрезков, содержащих точку $q_{x}$.
Пусть $I = \{[x_1 : x_{1}'], [x_2 : x_{2}'], \dots , [x_n : x_{n}']\}$ - множество отрезков на оси.
Возьмем уникальные концы этих отрезков и отсортируем их по возрастанию, получим точки $p_1, p_2, \dots , p_m$$(m\leqslant 2n)$.
Назовем множеством _элементарных_ _интервалов_ $E = \{ (-\infty : p_1), [p_1 : p_1], (p_1 : p_2),[p_2 : p_2], \dots , (p_{m-1} : p_{m}), [p_m : p_m],(p_m : +\infty) \}$ (см. рис.2). 

![](img/axis.svg)

Мы выделяем границы в отдельный интервал для того, чтобы уметь решать задачу с возможностью касания отрезков по границе.

Построим сбалансированное бинарное дерево $T$, листам которого будут соответствовать элементарные интервалы, а внутренним вершинам — объединения интервалов в потомках.
Будем обозначать $Int(u)$ интервал, соответствующий листу $u$. 

Пусть все отрезки из $I$, содержащиеся в $Int(u)$, хранятся в листе $u$.
Тогда мы можем найти $k$ отрезков, содержащих $q_{x}$, за время $O(\log(n) + k)$: находим за $O(\log(n))$ лист, содержащий $q_{x}$, затем за $O(k)$ возвращаем все отрезки из этого листа.
Отвечать мы можем эффективно, но что насчет потребляемой памяти?
Отрезки, которые перекрывают много элементарных интервалов, будут храниться во многих листах.
Значит, потребляемый объем будет расти, если есть много пар перекрывающихся отрезков.
В худшем случае будет достигнут квадратичный объем памяти.
Посмотрим, как мы можем это улучшить.   

![Общий предок](img/lca.svg)

Рассмотрим листы $u_1$, $u_2$, $u_3$, $u_4$ (см. рис.3).
Если $q_{x}$ принадлежит одному из этих интервалов, мы должны возвращать отрезок $s$.
Заметим, что спуск по дереву поиска заканчивается в любой из этих вершин тогда и только тогда, когда мы проходим через вершину $v$.
Поэтому давайте хранить $s$ в $v$ (и $u_5$) вместо $u_1$, $u_2$, $u_3$, $u_4$ (и $u_5$).
В общем случае мы храним отрезок в минимальном количестве вершин, для которых объединение соответствующих интервалов будет покрывать данный отрезок и только его. 

### Свойства:
Давайте теперь полностью опишем итоговые свойства нашей структуры.

* Каркасом структуры является бинарное сбалансированное дерево $T$.
  Листья $T$ соответствуют элементарным интервалам, индуцированным отсортированными границами $I$: самый левый лист - самому левому интервалу и тд.
* Внутренней вершине $v$ дерева $T$ соответствует интервал, являющийся объединением интервалов листов в поддереве, корнем которого она является.
  Иначе говоря, $Int(v)$ является объединением интервалов двух своих детей.
* Каждая вершина $v$ в $T$ хранит (например, в связном списке) в себе $Int(v)$ и множество отрезков $I(v) \subseteq I$, таких что: $\forall [x : x'] \in I(v) : Int(v) \subseteq [x : x'], Int(parent(v)) \not\subseteq [x : x']$. Назовем $I(v)$ каноническим подмножеством для вершины $v$.

На рисунке 4 представлен пример канонических множеств для соответствующего дерева. Чтобы не загромождать рисунок, для вершин, не содержащих отрезков, пустые списки опущены.  

![](img/tree.svg)

### Лемма (Оценка на память)
>Дерево отрезков занимает $O(n\log(n))$ памяти.

$\triangleright$
<div style="padding-left:40px">
Так как бинарное дерево сбалансировано и имеет не больше $4n + 1$ вершин, то его высота $O(\log(n))$. Утверждается, что любой отрезок $[x : x']\in I$ хранится не более, чем в двух вершинах на одной глубине дерева $T$. Действительно, пусть есть три вершины $v_1$, $v_2$, $v_3$ на одной глубине.
![](img/cross.svg)
Предположим, что $v_1$ и $v_3$ содержат $[x : x']$.
Это означает, что $[x : x']$ покрывает весь интервал от левой границы $Int(v_1)$ до правой границы $Int(v_3)$.
Так как $v_2$ лежит между $v_1$ и $v_3$, то $Int(parent(v_{2}))$ необходимо содержится в $[x : x']$, а значит, $[x : x']$ не должен храниться в $v_2$.
Противоречие. 
</div>
$\triangleleft$

### Построение дерева

Сначала мы сортируем точки границ отрезков из $I$ и получаем элементарные интервалы.
Затем строим по ним бинарное сбалансированное дерево и определяем для каждой вершины $v$ интервал $Int(v)$, который она представляет.
Далее определим для вершин их канонические подмножества: будем по очереди вставлять каждый отрезок из $I$, используя процедуру, описанную далее в виде псевдокода, запускать которую будем от корня дерева $T$.

$
\operatorname{InsertSegmentTree}(v,\;[x:x'])\text{:}\\
\quad \mathtt{if} \operatorname{Int}(v) \subseteq [x:x']\\
\qquad \operatorname{store}~[x:x']~\operatorname{at}~v\\
\quad\mathtt{else}\text{:}\\
\qquad\mathtt{if}\operatorname{Int}(\operatorname{LeftChild}(v)) \cap [x:x'] \neq \varnothing\\
\qquad\quad\operatorname{InsertSegmentTree}(\operatorname{LeftChild}(v),\;[x:x'])\\
\qquad\mathtt{if}\operatorname{Int}(\operatorname{RightChild}(v)) \cap [x:x'] \neq \varnothing\\
\qquad\quad\operatorname{InsertSegmentTree}(\operatorname{RightChild}(v),\;[x:x'])
$

В результате получим структуру данных, удовлетворяющую всем необходимым свойствам.

### Лемма (Оценка времени построения)
>Дерево отрезков можно построить за время $O(n\log(n))$.

$\triangleright$
<div style="padding-left:40px">
Сортировку точек можно выполнить за время $O(n\log(n))$.
Построить сбалансированное дерево и выставить соответствующие вершинам интервалы можно за линейное время, используя подъем снизу вверх (англ. bottom-up) от листьев, т.е. запуском от корня рекурсивного спуска с выставлением состояния вершине по ее детям перед выходом из нее.
Наибольший интерес представляет создание канонических подмножеств.
Сколько же занимает вставка отрезка $[x:x']$ в дерево?
Предполагая, что $I(v)$ мы храним в структуре наподобие связного списка, при посещении вершины мы тратим константное количество времени.
Оказавшись в вершине, мы либо сохраняем в ней $[x:x']$, либо ее интервал содержит один из концов $[x:x']$.
Выше мы уже доказывали, что на одной глубине дерева $T$ отрезок хранится не более чем в двух вершинах.
Так же на одной глубине не более одной вершины, чей интервал содержит $x$, аналогично с $x'$. 
Таким образом, на каждом уровне мы посещаем не более четырех вершин, а так как глубина дерева равна $\log(n)$, то вставка одного отрезка займет $O(\log(n))$.
А тогда суммарная асимптотика времени построения дерева отрезков будет $O(n\log(n))$.</div>
$\triangleleft$

### Упражнения
Для закрепления материала выполните небольшие упражнения.

#### Задание 1
Для построения дерева отрезков необходимо назначить вершинам элементарные интервалы, которые они покрывают.
Давайте здесь и займемся этим.
Вам необходимо написать функцию, которая для внутренних вершин определяет границы их элементарных интервалов. 

Указания: 
* Вам передается список вершин графа $graph$ и текущая вершина обработки $v$, которая при первом запуске равна 0, т.е. корню дерева. 
* Считайте, что для вершины с номером $i$ левым и правым потомками являются вершины, с номерами $2 \cdot i + 1$ и $2 \cdot i + 2$ соответственно.
* Для элементов массива $graph[i]$ определены следующие значения:
  * $graph[i][0]$ - левая граница интервала ($int$)
  * $graph[i][1]$ - правая  граница интервала ($int$)
  * $graph[i][2]$ - замкнутость\открытость левой границы интервала ($True\mid False$)
  * $graph[i][3]$ - замкнутость\открытость правой границы интервала ($True\mid False$)
* Для листьев границы уже определены, заполните остальные элементы дерева.

![](img/ex1.png)

In [ ]:
def build_tree(gr, i=0):
    # Напишите ваш код здесь.
    pass

# Проверка вашей процедуры    
building_tree_checker.test(build_tree)
# Визуализация ответа
IFrame("out/tree.pdf", width=900, height=400)

#### Задание 2
Легко заметить, что для всех внутренних вершин $v$ дерева отрезков $Int(v)$ всегда является открытым слева полуинтервалом, кроме самой правой ветки дерева.
Вам необходимо написать функцию, использующуюся для задания канонических подмножеств при построении дерева отрезков, которая определяет положение отрезка $\mathit{seg}$ относительно полуинтервал $\mathit{int}$.
Возвращаемое значение описывается перечислением $\mathit{Relation}$.
Если $\mathit{seg}\subset\mathit{int}$ функция должна возвращать $\mathit{Relation.CONTAINS}$, если $\mathit{seg}\cap\mathit{int}\ne\varnothing$, но $\mathit{seg}\not\subset\mathit{int}$, то $\mathit{Relation.INTERSECTS}$, и $\mathit{Relation.DOES\_NOT\_INTERSECT}$ иначе.

In [ ]:
def relation(segment, interval):
    # Рассматривайте параметры функции как массивы из двух элементов , например,
    # segment[0] - координата начала отрезка, segment[1] - координата конца отрезка.
    #
    # Считайте, что координаты концов всегда корректные.
    #
    # Напишите ваш код здесь.
    pass

relation_checker.test(relation)

### Лемма (Запрос в дереве отрезков)
>С помощью дерева отрезков можно отвечать на запросы о поиске всех отрезков, содержащих точку $q_{x}$, за время $O(\log(n) + k)$, где k - количество отрезков в ответе.

$\triangleright$
<div style="padding-left:40px">
Запрос о поиске отрезков, которые содержат точку $q_{x}$ можно описать следующим псевдокодом процедуры, которая запускается от корня и точки запроса:

$
\operatorname{QuerySegmentTree}(v,\;q_{x})\text{:}\\
\quad\operatorname{print~all~intervals~in}\operatorname{Int}(v)\\
\quad\mathtt{if} \operatorname{v~is~not~a~leaf}\text{:}\\
\qquad\mathtt{if}~q_{x}~\in \operatorname{Int}(\operatorname{LeftChild}(v))\text{:}\\
\qquad\quad \operatorname{QuerySegmentTree}(\operatorname{LeftChild}(v),\;q_{x})\\
\qquad\mathtt{else}\text{:}\\
\qquad\quad \operatorname{QuerySegmentTree}(\operatorname{RightChild}(v),\;q_{x})\\
$

На каждом уровне дерева $T$ алгоритм посещает ровно одну вершину, поэтому время на обход вершин $O(\log(n))$.
Для каждой вершины $v$ мы тратим $O(k_{v})$ времени, где $k_{v}$ - количество искомых в отрезков в вершине $v$, чтобы сообщить об этих отрезках.
Откуда и получается суммарное время $O(\log(n) + k)$, где k - количество отрезков в ответе.</div>
$\triangleleft$

Собирая вместе полученные выше результаты, приходим к следующей теореме.  

### Теорема (О поиске всех отрезков, содержащих точку)
>Дерево отрезков для множества $I$ из $n$ отрезков можно построить за время $O(n\log(n))$, и оно будет занимать $O(n\log(n))$ памяти. 
 Используя его, можно находить все отрезки, содержащие точку $q_{x}$, за время $O(\log(n) + k)$, где k - количество отрезков в ответе.

## Применение дерева отрезков

Вернемся теперь к заявленной в самом начале задаче.
Пусть $S$ - множество случайно направленных непересекающихся отрезков на плоскости.
Мы хотим уметь находить все отрезки, которые пересекают вертикальный отрезок $q = q_{x}\times[q_{y}:q_{y}']$.
Для горизонтального отрезка $q = [q_{x}:q_{x}']\times q_{y}$ задача будет решаться по аналогии.

Давайте посмотрим, что мы получим, если построим дерево отрезков на проекциях на ось $Ox$ отрезков из $S$.
Вершина $v$ в дереве $T$ может быть интерпретирована как вертикальная полоса $Int(v)\times(-\infty:+\infty)$.
Пусть отрезок полностью пересекает вертикальную полосу, соответствующую вершине $v$, но полосу родителя вершины $v$ он полностью не пересекает, тогда он входит в каноническое подмножество для вершины $v$, обозначим его $S(v)$. 

![](img/band.svg)

Когда мы ищем $q_{x}$ в $T$, мы находим $O(\log(n))$ канонических подмножеств из тех вершин, которые нам встречаются на пути.
Вместе они образуют множество всех отрезков, проекции на ось $Ox$ которых содержат точку $q_{x}$.
В пределах одной вертикальной полосы для вершины $v$ мы можем воспользоваться тем фактом, что отрезки из канонического подмножества не пересекаются и полностью перекрывают полосу, откуда следует возможность задания вертикального порядка на элементах канонического подмножества.
Все отрезки между самым верхним и самым нижним отрезками, пересекающими $q$, тоже обязательно пересекают $q$. Поэтому будем хранить $S(v)$ в сбалансированном дереве поиска  $\tau(v)$ на вертикальном порядке (см. рис.7).
Таким образом, для вершины $v$ поиск отрезков, пересекающих $q$, занимает $O(\log(n) + k_{v})$ времени, где $k_{v}$ - количество пересекающих отрезков.

![](img/vert_heap.svg)

### Свойства
Резюмируем требуемые от нашей структуры данных свойства:
* Множество $S$ хранится в дереве отрезков, основанном на горизонтальных проекциях отрезков из него.
* Каноническое подмножество для вершины $v$ (состоящее из отрезков, которые полностью перекрывают вертикальную полосу для $v$, но не для ее родителя) хранится в сбалансированном дереве поиска $\tau(v)$, основанном на вертикальном порядке в пределах вертикальной полосы. 

### Оценка на время построения

Так как ассоциированное с каждой вершиной дерево поиска занимает линейное по отношению к $S(v)$ количество памяти, то суммарные затраты на память по-прежнему $O(n\log(n)$.
Для каждой вершины дерево может быть построено за $O(n\log(n)$, откуда следует увеличение времени препроцессинга до $O(n\log^2(n))$.
Однако, если задать частичный вертикальный порядок на отрезках во время построения дерева отрезков, то можно сохранить прежнюю суммарную асимптотику $O(n\log(n))$.  

### Выполнение запросов

Алгоритм для поиска всех пересекающих $q$ отрезков довольно прост: как и в предыдущей задаче мы ищем $q_{x}$ в дереве отрезков, но теперь в каждой вершине $v$ на пути находим верхнюю и нижнюю границы (англ. lower and upper bound) пересечения с $q$, и возвращаем отрезки $S(v)$ из этого промежутка.     

#### Оценка на время

Поиск в $\tau(v)$ занимает $O(\log(n) + k_{v})$ времени, где $k_{v}$ - количество отрезков для $v$.
Таким образом, суммарное время на запрос $O(\log^2(n) + k)$.

Собирая вместе полученные факты, сформулируем несколько теорем.

### Теорема (О пересечении вертикального (горизонтального) отрезка с множеством непересекающихся отрезков)
>Пусть $S$ - множество непересекающихся отрезков на плоскости.
Отрезки, пересекающие заданный вертикальный отрезок, могут быть найдены за время $O(\log^2(n) + k)$ структурой данных, использующей $O(n\log(n))$ памяти, где $k$ - количество отрезков в ответе.
Структура данных может быть построена за $O(n\log(n))$ времени.

Такой же подход можно применить к отрезкам, которые имеют непересекающиеся внутренности, но могут иметь общие границы.

### Теорема (О пересечении прямоугольника с множеством непересекающихся по внутренностям отрезков)
>Пусть $S$ - множество отрезков на плоскости с непересекающимися внутренностями.
Отрезки, пересекающие прямоугольник, стороны которого параллельны осям координат, могут быть найдены за время $O(\log^2(n) + k)$ структурой данных, использующей $O(n\log(n))$ памяти, где $k$ - количество отрезков в ответе. Структура данных может быть построена за $O(n\log(n))$ времени.

$\triangleright$
<div style="padding-left:40px">
Для каждой стороны прямоугольника воспользуемся предыдущей теоремой.
Никакие оценки не изменятся, поэтому получим требуемый результат.
</div>
$\triangleleft$